In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import keras

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from joblib import dump, load
from tensorflow.keras.callbacks import ModelCheckpoint


from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras import optimizers

import matplotlib.pyplot as plt



In [ ]:
data_start = pd.read_csv('../input/etudataset/data.csv',  sep=',')
marks = pd.read_csv('../input/etudataset/avg_marks.csv',  sep=',')


In [ ]:
data = data_start[['БАЛЛ_ЗА_ДОСТИЖЕНИЯ', 'БАЛЛ_ОЛИМПИАДЫ_И_КОНКУРСЫ', 'ЕСТЬ_АТТЕСТАТ_С_ОТЛИЧИЕМ', 'ЕСТЬ_ДИПЛОМ_С_ОТЛИЧИЕМ','ОЦЕНКА_ЗА_СОЧИНЕНИЕ','УРОВЕНЬ_ДИПЛОМА','ОБЩЕЖИТИЕ_ТИП_ЗАСЕЛЕНИЯ','НАПРАВЛЕНИЕ_В_ПРИКАЗЕ_БАЛЛ','НАПРАВЛЕНИЕ_В_ПРИКАЗЕ_БАЛЛ_ЕГЭ_С_ОЛИМПИАДОЙ','НАПРАВЛЕНИЕ_В_ПРИКАЗЕ_ОЛИМПИАДА_ЗА_100_БАЛЛОВ','НАПРАВЛЕНИЕ_В_ПРИКАЗЕ_ПРЕДМЕТ_1_ПРИОРИТЕТ','НАПРАВЛЕНИЕ_В_ПРИКАЗЕ_ПРЕДМЕТ_2_ПРИОРИТЕТ','НАПРАВЛЕНИЕ_В_ПРИКАЗЕ_ПРЕДМЕТ_3_ПРИОРИТЕТ','НАПРАВЛЕНИЕ_В_ПРИКАЗЕ_ОЦЕНКА_1','НАПРАВЛЕНИЕ_В_ПРИКАЗЕ_ОЦЕНКА_2','НАПРАВЛЕНИЕ_В_ПРИКАЗЕ_ОЦЕНКА_3','НАПРАВЛЕНИЕ_ПОДГОТОВКИ','ИНДИВИДУАЛЬНЫЙ_НОМЕР']]
data = data.rename({'ИНДИВИДУАЛЬНЫЙ_НОМЕР' : 'keyID'}, axis = 1)

In [ ]:

x = data['ОБЩЕЖИТИЕ_ТИП_ЗАСЕЛЕНИЯ'].unique()
sub = data['НАПРАВЛЕНИЕ_В_ПРИКАЗЕ_ПРЕДМЕТ_2_ПРИОРИТЕТ'].unique()


In [ ]:
subject = {'Математика': 0, 'Английский язык': 1, 'Информатика': 2,'Физика': 3, 'Обществознание': 4, 'Русский язык': 5, 'Химия': 6}
education = {'Среднее': 0, 'Начальное профессиональное': 1, 'Среднее профессиональное': 2 , 'Высшее профессиональное': 3}
loc_student = {'Общежитие': 1, 'Только регистрация': 2}
list_cours = {"10.05.01": 0,
"27.03.02": 1,
"42.03.01": 2,
"28.03.01": 3,
"45.03.02": 4,
"12.03.04": 5,
"38.03.02": 6,
"27.03.03": 7,
"09.03.04": 8,
"20.03.01": 9,
"38.03.01": 10,
"11.03.02": 11,
"27.03.04": 12,
"11.03.03": 13,
"09.03.02": 14,
"13.03.02": 15,
"12.03.01": 16,
"09.03.01": 17,
"01.03.02": 18,
"27.03.05": 19,
"11.03.04": 20,
"11.03.01": 21}

prioritet = ['НАПРАВЛЕНИЕ_В_ПРИКАЗЕ_ПРЕДМЕТ_1_ПРИОРИТЕТ','НАПРАВЛЕНИЕ_В_ПРИКАЗЕ_ПРЕДМЕТ_2_ПРИОРИТЕТ','НАПРАВЛЕНИЕ_В_ПРИКАЗЕ_ПРЕДМЕТ_3_ПРИОРИТЕТ']

for example in education:
    data['УРОВЕНЬ_ДИПЛОМА'] = np.where(data['УРОВЕНЬ_ДИПЛОМА'] == example, education[example], data['УРОВЕНЬ_ДИПЛОМА']) 
    
for example in loc_student:
    data['ОБЩЕЖИТИЕ_ТИП_ЗАСЕЛЕНИЯ'] = np.where(data['ОБЩЕЖИТИЕ_ТИП_ЗАСЕЛЕНИЯ'] == example, loc_student[example], data['ОБЩЕЖИТИЕ_ТИП_ЗАСЕЛЕНИЯ'])  
data['ОБЩЕЖИТИЕ_ТИП_ЗАСЕЛЕНИЯ'] = data['ОБЩЕЖИТИЕ_ТИП_ЗАСЕЛЕНИЯ'].replace(np.nan, 0) 

for attr in prioritet:
    for example in subject:
        data[attr] = np.where(data[attr] == example, subject[example], data[attr]) 

        
data['НАПРАВЛЕНИЕ_ПОДГОТОВКИ'] = data['НАПРАВЛЕНИЕ_ПОДГОТОВКИ'].str[:8]
#for example in list_cours:
    #data['НАПРАВЛЕНИЕ_ПОДГОТОВКИ'] = np.where(data['НАПРАВЛЕНИЕ_ПОДГОТОВКИ'] == example, list_cours[example], data['НАПРАВЛЕНИЕ_ПОДГОТОВКИ']) 
    

In [ ]:
if ((len(marks))!=(len(marks['keyID'].unique()))):
    duplicateRows_Labels=marks[marks.duplicated(['keyID'],keep=False)]
    for x in range(len(duplicateRows_Labels)):
        marks.drop(duplicateRows_Labels.index[x],inplace=True)

In [ ]:
data

In [ ]:
all_data = pd.merge(data, marks, on='keyID', right_index = False ,sort=False)

In [ ]:
#all_data = all_data.loc[all_data['avg_mark'] != 0]
#all_data = all_data.loc[all_data['avg_mark'] != 2]
all_data = all_data.loc[all_data['avg_mark'] > 2.9]

In [ ]:
all_data = all_data[all_data['НАПРАВЛЕНИЕ_В_ПРИКАЗЕ_БАЛЛ'] != 0]

In [ ]:

x = all_data
x = x.drop(['НАПРАВЛЕНИЕ_В_ПРИКАЗЕ_БАЛЛ_ЕГЭ_С_ОЛИМПИАДОЙ','НАПРАВЛЕНИЕ_В_ПРИКАЗЕ_ОЛИМПИАДА_ЗА_100_БАЛЛОВ', 'avg_mark','keyID'], axis = 1)

print('Dividing data...')
train_data = np.array(x.drop('НАПРАВЛЕНИЕ_ПОДГОТОВКИ', axis = 1)).astype(np.float)
train_label = np.array(x['НАПРАВЛЕНИЕ_ПОДГОТОВКИ']).astype(np.int)

norm = Normalizer()

print('Preprocessing data...')
train_data = norm.fit_transform(train_data)

train_X, test_X, train_y, test_y = train_test_split(train_data, train_label, test_size=0.2, random_state=1)


from tensorflow.keras import utils
train_y = utils.to_categorical(train_y, 22)
test_y = utils.to_categorical(test_y, 22)

model = Sequential()
model.add(Dense(22, activation='tanh', input_shape = (train_X.shape[1],)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(256, activation='tanh'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(512, activation='tanh'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(22, activation = 'softmax'))

opt = optimizers.SGD(learning_rate=0.01)

model.compile(optimizer=opt, loss = 'categorical_crossentropy', metrics=['accuracy'])


print('Fitting model...')
history = model.fit(train_X, train_y, epochs=7, batch_size=5, validation_split = 0.05)





In [ ]:
score, acc = model.evaluate(test_X, test_y,
                            batch_size=1)

In [ ]:
for i in range(test_y.shape[0]):
    if (np.argmax(test_y[i]) == np.argmax(pred[i])):
        print('Succes!!', np.argmax(test_y[i]), np.argmax(pred[i]))
    else:
        print(np.argmax(test_y[i]), np.argmax(pred[i]))

In [ ]:
course = all_data['НАПРАВЛЕНИЕ_ПОДГОТОВКИ'].unique()

In [ ]:
course

In [ ]:
 with open("LR_results_update_v4.csv", "a") as file:
    # Writing data to a file
        file.writelines(['course,mse,mae'])
        file.close()

for example in course:

    x = all_data.loc[all_data['НАПРАВЛЕНИЕ_ПОДГОТОВКИ'] == example ]
    x = x.drop(['НАПРАВЛЕНИЕ_ПОДГОТОВКИ','НАПРАВЛЕНИЕ_В_ПРИКАЗЕ_БАЛЛ_ЕГЭ_С_ОЛИМПИАДОЙ','НАПРАВЛЕНИЕ_В_ПРИКАЗЕ_ОЛИМПИАДА_ЗА_100_БАЛЛОВ', 'keyID'], axis = 1)
    print('#################################')
    print('INFO:')
    print('Training ', example, ' course..')
    print('#################################')
    
    print('Dividing data...')
    train_data = np.array(x.drop('avg_mark', axis = 1))
    train_label = x['avg_mark']
    
   
    #norm = Normalizer()
    
    print('Preprocessing data...')
    #train_data = norm.fit_transform(train_data)
    

    #lr=LinearRegression()

    # create the RFE model for the svm classifier 
    # and select attributes
    #rfe = RFE(lr, 10)
    #rfe = rfe.fit(train_data, train_label)

    # print summaries for the selection of attributes
    #print(rfe.support_)
    #print(rfe.ranking_)
    
    print('Spliting data for test / train ...')
    train_X, test_X, train_y, test_y = train_test_split(train_data, train_label, test_size=0.1, random_state=1)
    
    train_y = np.array(train_y)
    test_y = np.array(test_y)
    
    
    model = LinearRegression()
    
    print('Fitting model...')
    model.fit(train_X, train_y)
    
    print('Model score: ', model.score(train_X, train_y))
    
    print('Fitting model with test data...')
    y_pred = model.predict(test_X)
    
    
    mse = mean_squared_error(test_y, y_pred)
    mae = mean_absolute_error(test_y, y_pred)
    print('Score on test data: mse: %.3f, mae: %.3f' % (mse, mae))
   
  
 
    with open("LR_results_update_v4.csv", "a") as file:
    # Writing data to a file
        file.writelines([str(example) + ',' + str(mse) +',' + str(mae) + '\n'])
        file.close()
    
    
    filename = example + '-model.joblib'
    print('Saving model...')
    dump(model, filename)
    print('END INFO')
    print('#################################')
    print()

In [ ]:
all_data

In [ ]:
def checkpoint(filename):

    checkpoint_callback = ModelCheckpoint(filename, 
                                      monitor='val_mse',
                                      save_best_only=True,
                                      verbose=1)
    return checkpoint_callback

In [ ]:
with open("LR_results_update_v5.csv", "a") as file:
    # Writing data to a file
    file.writelines(['course,mse,mae'])
    file.close()

for example in course:
    x = all_data.loc[all_data['НАПРАВЛЕНИЕ_ПОДГОТОВКИ'] == example]
    x = x.drop(['НАПРАВЛЕНИЕ_ПОДГОТОВКИ', 'НАПРАВЛЕНИЕ_В_ПРИКАЗЕ_БАЛЛ_ЕГЭ_С_ОЛИМПИАДОЙ',
                'НАПРАВЛЕНИЕ_В_ПРИКАЗЕ_ОЛИМПИАДА_ЗА_100_БАЛЛОВ', 'keyID'], axis=1)

    print('#################################')
    print('INFO:')
    print('Training ', example, ' course..')
    print('#################################')

    filename = example + '-model'
    
    checkpoint_callback = checkpoint(filename)
    
    print('Dividing data...')
    train_data = np.array(x.drop('avg_mark', axis=1))
    train_label = x['avg_mark']

    norm = Normalizer()

    print('Preprocessing data...')
    train_data = norm.fit_transform(train_data)

    train_X, test_X, train_y, test_y = train_test_split(train_data, train_label, test_size=0.3, random_state=1)
    test_X = np.asarray(test_X).astype(np.float)
    train_X = np.asarray(train_X).astype(np.float)
    print(train_X.shape)
    train_y = np.asarray(train_y).astype(np.float)
    test_y = np.asarray(test_y).astype(np.float)


    model = Sequential()
    model.add(Dense(22, activation='relu', input_shape = (train_X.shape[1],)))
    model.add(Dropout(.2))
    model.add(Dense(22, activation='relu'))
    model.add(Dropout(.2))
    model.add(Dense(11, activation='relu'))
    model.add(Dropout(.2))
    model.add(Dense(1))

    opt = optimizers.SGD(learning_rate=0.001)

    model.compile(optimizer=opt, loss = 'mae', metrics=['mse'])

    
    print('Fitting model...')
    history = model.fit(train_X, train_y, epochs=20,validation_split = 0.05, batch_size=1,callbacks=[checkpoint_callback])
    
    mae, mse = model.evaluate(test_X, test_y)
    print('MSE on test data : ', mse)
    print('MAE on test data : ', mae)
    
    with open("LR_results_update_v5.csv", "a") as file:
    # Writing data to a file
        file.writelines([str(example) + ',' + str(mse) +',' + str(mae) + '\n'])
        file.close()
    #print('Saving model ...')
    filename = example + '-model'
    #model.save(filename)
    
    
    
    print('END INFO')
    print('#################################')
    print()

In [ ]:
train_X

In [ ]:
x = all_data.loc[all_data['НАПРАВЛЕНИЕ_ПОДГОТОВКИ'] == '09.03.01']
x = x.drop(['НАПРАВЛЕНИЕ_ПОДГОТОВКИ', 'НАПРАВЛЕНИЕ_В_ПРИКАЗЕ_БАЛЛ_ЕГЭ_С_ОЛИМПИАДОЙ',
                'НАПРАВЛЕНИЕ_В_ПРИКАЗЕ_ОЛИМПИАДА_ЗА_100_БАЛЛОВ', 'keyID'], axis=1)
train_data = np.array(x.drop('avg_mark', axis=1))
train_label = x['avg_mark']



#train_data = norm.fit_transform(train_data)

In [ ]:
train_label[1]

In [ ]:
model_test = keras.models.load_model('09.03.01-model')

In [ ]:
train_data = np.array(train_data).astype(float)
train_label = np.array(train_label).astype(float)

In [ ]:
train_data

In [ ]:
train_label[879]

In [ ]:
test = [[0.0, 0.0, 0.0, 0.0, 0.0, 0, 0, 222.0, 0, 2, 5, 62.0, 73.0, 87.0]]
test = norm.fit_transform(test)
print(test)

pred = model_test.predict(test)


In [ ]:
pred

In [ ]:
for i in range(pred.shape[0]):
    print(train_label[i], ' pred:', pred[i])

In [ ]:
train_y[2]

In [ ]:
train_data

In [ ]:
mse, mae = model.evaluate(test_X, test_y)

In [ ]:
pred = model.predict(train_X)

In [ ]:
pred

In [ ]:
test_data

In [ ]:
from sklearn.preprocessing import Normalizer
sc = Normalizer()
test_data = np.array([[0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 5, 67, 60, 60]])
test_data = sc.fit_transform(test_data)

test_data.shape

In [ ]:
pred = model.predict(test_data)

In [ ]:
model

In [ ]:
test_y

In [ ]:
count = 0
for i in range(pred.shape[0]):
    if (abs(test_y[i]-pred[i])) < 0.5:
        print(pred[i], test_y[i],'diff: ', abs(test_y[i]-pred[i]))
        count+=1
print(count)

In [ ]:
x = all_data.loc[all_data['НАПРАВЛЕНИЕ_ПОДГОТОВКИ'] == '28.03.01' ]
x = x.drop(['НАПРАВЛЕНИЕ_ПОДГОТОВКИ','НАПРАВЛЕНИЕ_В_ПРИКАЗЕ_БАЛЛ_ЕГЭ_С_ОЛИМПИАДОЙ','НАПРАВЛЕНИЕ_В_ПРИКАЗЕ_ОЛИМПИАДА_ЗА_100_БАЛЛОВ', 'keyID'], axis = 1)

model = LinearRegression(normalize = True)
train_data = np.array(x.drop('avg_mark', axis = 1))
train_label = x['avg_mark']
    

    
print('Spliting data for test / train ...')
train_X, test_X, train_y, test_y = train_test_split(train_data, train_label, test_size=0.1, random_state=1)
   
train_y = np.array(train_y)
test_y = np.array(test_y)



In [ ]:
model.fit(train_X, train_y)


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
sc = MinMaxScaler()
sn = Normalizer()
test_data = np.array([[0, 0, 0, 11111111110, 0, 2, 0, 0, 0, 2, 5, 67, 60, 60], [6,5,5,5,5,5,5,5,5,7,3,90,100,50]])
test_data = sc.fit_transform(test_data)
test_data = sn.fit_transform(test_data)

In [ ]:
test_data = [[ 0  ,0 , 0 , 0 , 0 , 0 , 0 , 0 , 1 , 4 , 5, 80 ,80, 80]]
#test_data = sn.fit_transform(test_data)

In [ ]:
test_data

In [ ]:
pred = model.predict(test_data)

In [ ]:
pred


In [ ]:
model.score(train_X, train_y)